In [2]:
import numpy as np
import torch 

In [4]:
t1 = torch.tensor(3)
t1

tensor(3)

In [5]:
t2= torch.tensor((1,2,3,4))
t2


tensor([1, 2, 3, 4])

In [6]:
t2.dtype

torch.int64

In [7]:
t2.shape

torch.Size([4])

In [8]:
# Matrix 
t5 = torch.tensor(([1,2],[3,4]))
t5

tensor([[1, 2],
        [3, 4]])

In [9]:
t5.shape

torch.Size([2, 2])

In [10]:
x = torch.tensor(([2,3],[4,6],[8,2])




SyntaxError: unexpected EOF while parsing (<ipython-input-10-c5cb275aedfa>, line 3)

In [11]:
c = torch.tensor(([1.,2.],[4.,4.]), requires_grad =True)
d= torch.tensor(([6.,7.],[8.,8.]), requires_grad = True)

In [12]:
y  = c + d
y

tensor([[ 7.,  9.],
        [12., 12.]], grad_fn=<AddBackward0>)

In [13]:
#Compute derivatives 
# Can't use matrix for backward prop using .backward()


In [14]:
w

NameError: name 'w' is not defined

In [ ]:
w.grad   #dy/dw

In [15]:
b.grad   #dy/db

NameError: name 'b' is not defined

In [16]:
a = np.array(([1,2],[6,7]))
a

array([[1, 2],
       [6, 7]])

In [17]:
#Convert array to tensor 
b = torch.tensor(a) # This creates a copy
b

tensor([[1, 2],
        [6, 7]], dtype=torch.int32)

In [18]:
c = torch.from_numpy(a) #Refers to same space in the memory
c

tensor([[1, 2],
        [6, 7]], dtype=torch.int32)

In [19]:
b.dtype, c.dtype

(torch.int32, torch.int32)

In [20]:
o = b.numpy()  # Conversion back to numpy
o

array([[1, 2],
       [6, 7]])

In [21]:
# Linear regression 
# input (temp, rainfall, humidity)

inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

In [22]:
# Targets (apples, oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype = 'float32')

In [23]:
# Convert inputs and targets into tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [24]:
# Linear regresssion requires weights and biases 
w = torch.randn(2,3, requires_grad=True)
b = torch.randn(1,requires_grad = True)
w

tensor([[ 0.1341, -2.4455,  2.4354],
        [-0.4722, -0.6837,  0.3503]], requires_grad=True)

In [25]:
b

tensor([0.5164], requires_grad=True)

In [26]:
def model(x):
    return x@ w.T +b

In [27]:
# Generate predictions 
pred = model(inputs)
print(pred)

tensor([[ -48.8213,  -64.6982],
        [ -46.6201,  -80.1986],
        [-174.2607, -111.8621],
        [  -0.8532,  -64.0852],
        [ -54.5218,  -73.1777]], grad_fn=<AddBackward0>)


In [28]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Huge difference between predictions and target 


### Now we need to use gradient descent to optimize the function 

## Loss function 


In [29]:
# MSE loss 
diff = pred - targets
diff_squared = diff*diff
torch.sum(diff_squared)/ diff.numel() # numel is the number of elements 

tensor(29669.6055, grad_fn=<DivBackward0>)

In [30]:
def mse(x,y):
    diff = x-y    
    return torch.sum(diff*diff)/ diff.numel()

In [31]:
# compute the loss now 
loss = mse(pred,targets)
print(loss)

tensor(29669.6055, grad_fn=<DivBackward0>)


## Compute gradients 

In [32]:
loss.backward()

In [33]:
print(w)
print(w.grad)

tensor([[ 0.1341, -2.4455,  2.4354],
        [-0.4722, -0.6837,  0.3503]], requires_grad=True)
tensor([[-11595.8184, -14731.0615,  -8311.2432],
        [-14239.1992, -16115.5000,  -9756.6650]])


In [34]:
print(b)
print(b.grad)

tensor([0.5164], requires_grad=True)
tensor([-312.0198])


In [35]:
w.grad.zero_()
b.grad.zero_()
print(w.grad.zero_())
print(b.grad.zero_())

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0.])


## Steps
1. Generate predictions 
2. Calculate the loss 
3. Compute the gradients w.r.t the weights and biases 
4. Adjust the weights by subtracting a small quantity proportional to the gradient 
5. Reset the gradients to zero 


In [36]:
pred = model(inputs)
print(pred)

tensor([[ -48.8213,  -64.6982],
        [ -46.6201,  -80.1986],
        [-174.2607, -111.8621],
        [  -0.8532,  -64.0852],
        [ -54.5218,  -73.1777]], grad_fn=<AddBackward0>)


In [37]:
# calc the loss 
loss = mse(pred, targets)
print(loss)

tensor(29669.6055, grad_fn=<DivBackward0>)


In [38]:
# Compute gradients 
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-11595.8184, -14731.0615,  -8311.2432],
        [-14239.1992, -16115.5000,  -9756.6650]])
tensor([-312.0198])


In [40]:
# Adjust the weights and reset gradients 
with torch.no_grad():
    w -=w.grad*1e-5
    b -= b.grad*1e-5
    w.grad.zero_()
    b.grad.zero_()

In [41]:
print(w)
print(b)

tensor([[ 0.2501, -2.2982,  2.5185],
        [-0.3298, -0.5225,  0.4479]], requires_grad=True)
tensor([0.5195], requires_grad=True)


In [44]:
pred = model(inputs)
loss = mse(pred, targets)
print(loss)

tensor(20760.3594, grad_fn=<DivBackward0>)


## Train for mutliple epochs(passes)


In [56]:
# For 100 epochs 
for i in range(1000):
    pred = model(inputs)
    loss = mse(pred, targets)
    loss.backward()
    with torch.no_grad():
        w -=w.grad*1e-5
        b -= b.grad*1e-5
        w.grad.zero_()
        b.grad.zero_()
        

In [57]:
# Calculate the loss 
pred = model(inputs)
loss = mse(pred, targets)
print(loss)

tensor(17.0396, grad_fn=<DivBackward0>)


In [58]:
pred

tensor([[ 56.7673,  70.3832],
        [ 86.2124, 100.7083],
        [110.2244, 132.7540],
        [ 18.7075,  36.9528],
        [110.3740, 119.3405]], grad_fn=<AddBackward0>)

In [59]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [60]:
loss


tensor(17.0396, grad_fn=<DivBackward0>)